In [128]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [11]:
cd  "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [12]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring ='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [142]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_colors_cat'] = df['colors'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_categories_cat'] = df['categories'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [24]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [27]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [39]:
test = {'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [74]:
str_dict = '[{"key":"Gender","value":["Men"]}, {"key":"Shoe Size", "value":["M"]}, {"key":"Shoe Category","value":["Men\'s Shoes"]}, {"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]}, {"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0]

  output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [75]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys() ) )

len(keys)

159

In [78]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [81]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_inseam', 'feat_front', 'feat_multi pack indicator',
       'feat_variant group id', 'feat_occasion', 'feat_band material',
       'feat_labor', 'feat_color mapping', 'feat_certifications and listings',
       'feat_product type'],
      dtype='object', length=209)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [85]:
df.shape[0]

18280

In [97]:
{k:v for k,v in keys_stat.items() if v > 10}

{'age group': 13.867614879649889, 'shoe width': 14.157549234135667}

In [98]:
df [ df['brand'] == 'nike'].features_parsed.head().values

array([{'condition': 'New with box'}, {'condition': 'New with box'},
       {'type': 'Cleats'}, {'condition': 'New without box'},
       {'type': 'Cleats'}], dtype=object)

In [0]:
feats = ['brand_cat', 'feat_colors_cat', 'feat_categories_cat']
result = run_model(feats, model)

In [144]:
X = df[ feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-58.8856316958883, 4.442653878430622)


Weight,Feature
0.2119 ± 0.0097,brand_cat
0.1158 ± 0.0034,feat_categories_cat
0.0919 ± 0.0036,feat_colors_cat


In [146]:
!git add day5.ipynb

fatal: pathspec 'day5.ipynb' did not match any files


In [148]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix
